# Install

To install, after creating conda environment from `environment.yml` run in R:

```R
install.packages("jackalope", type="source")
```

Note, please see <https://lucasnell.github.io/jackalope/index.html#enabling-openmp> for enabling openmp to use multiple threads.

# Simulate reads

This simulates mutations and reads of a small dataset for testing purposes using a reference genome and a tree.

In [1]:
library(jackalope)

# Make sure we've complied with openmp
jackalope:::using_openmp()

[1] TRUE

In [2]:
reference <- read_fasta("input/genome.fasta.gz")
reference

< Set of 1 chromosomes >
# Total size: 5,180 bp
  name                             chromosome                             length
reference  AGAGATTACGTCTGGTTGCAAGAGATCA...GAGGTTTCTTCATCAAAGAAATACCCTT      5180

In [3]:
library(ape)

tree <- read.tree("input/tree.tre")
tree <- root(tree, "SampleA", resolve.root=TRUE)
tree


Phylogenetic tree with 4 tips and 3 internal nodes.

Tip labels:
  reference, SampleA, SampleB, SampleC

Rooted; includes branch lengths.

In [4]:
sub <- sub_JC69(lambda = 1e-3, mu = 1)
ins <- indels(rate = 1e-3, max_length = 60,a = 1.60)
del <- indels(rate = 1e-3, max_length = 60, a = 1.51)

ref_haplotypes <- create_haplotypes(reference, haps_phylo(tree), sub=sub, ins=ins, del=del)
ref_haplotypes

                              << haplotypes object >>
# Haplotypes: 4
# Mutations: 144

                          << Reference genome info: >>
< Set of 1 chromosomes >
# Total size: 5,180 bp
  name                             chromosome                             length
reference  AGAGATTACGTCTGGTTGCAAGAGATCA...GAGGTTTCTTCATCAAAGAAATACCCTT      5180

In [5]:
write_vcf(ref_haplotypes, out_prefix="output/haplotypes", compress=TRUE)

In [6]:
coverage <- 30
read_length <- 250
n_samples <- length(tree$tip)
n_reads <- round((reference$sizes() * coverage * n_samples) / read_length)
cat("Number of reads:", n_reads)

illumina(ref_haplotypes, out_prefix = "output/reads/initial", sep_files=TRUE, n_reads = n_reads,
         frag_mean = read_length * 2 + 50, frag_sd = 100,
         compress=TRUE, comp_method="bgzip", n_threads=48,
         paired=TRUE, read_length = read_length)

Number of reads: 2486

# Take care of reference reads

In order to compare trees, I need to remove reads simulated from reference genome (so reference isn't included twice).

First, I verify that the simulated reference is identical to the input reference genome.

In [7]:
out <- system("zcat output/haplotypes.vcf.gz | tail -n+9 | cut -f 10 | sort -u", intern=TRUE)
print(out)

[1] "0:441453"  "reference"


What this does is counts the unique values of the genotype column from the VCF containing all simulated variants. The `reference` means this is the reference genome column. The `0:` part indicates that the variant in this particular position is identical to the reference genome (`1` and `2`, ... get used for alternative alleles). So, here we can see that the only value is `0:` for the reference genome (hence all variants are identical to it).

To see what this looks like for another genome, let's try:

In [8]:
out <- system("zcat output/haplotypes.vcf.gz | tail -n+9 | cut -f 11 | sort -u", intern=TRUE)
print(out)

[1] "0:441453" "1:441453" "SampleA" 


This indicates `SampleA` has some alternative alleles in the VCF file.

So, we want to remove the reads for the simulated reference genome.

In [9]:
# Remove the simulated reads for the reference genome since I don't want these in the tree
ref1 <- "output/reads/initial_reference_R1.fq.gz"
ref2 <- "output/reads/initial_reference_R2.fq.gz"
if (file.exists(ref1)) {
    file.remove(ref1)
}
if (file.exists(ref2)) {
    file.remove(ref2)
}

[1] TRUE

[1] TRUE